In [44]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import butter, filtfilt
import pyedflib
from scipy.signal import welch
from scipy import stats
from tqdm import tqdm

In [6]:
# function for reading .edf data
def edf_to_dataframe(edf_file_path):
    # Reading the EDF file
    f = pyedflib.EdfReader(edf_file_path)
    
    # Extracting signal labels
    signal_labels = f.getSignalLabels()
    
    # Initializing a dictionary to store signals
    signals_dict = {}
    
    # Extract each signal and store in the dictionary
    for i, label in enumerate(signal_labels):
        signals_dict[label] = f.readSignal(i)
    
    # Close the EDF file
    f.close()
    
    # Convert dictionary to DataFrame
    df = pd.DataFrame(signals_dict)
    
    return df

In [10]:
# reading all the data files
import os

names = []
dfs = []
labels = [] # 0 = control 1 = knockout

folder_path = 'All_Spontaneous'
files = os.listdir(folder_path)

for file_name in files:
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path):  # Ensure it's a file
        with open(file_path, 'r') as file:
            if '.edf' in file_path:
                try:
                    dfs.append(edf_to_dataframe(file_path))
                    names.append(file_name)                
                    if 'KO' in file_name :
                        labels.append(1)
                    else:
                        labels.append(0)
                except Exception as e:
                    print(f"Error reading the file: {e}")

Error reading the file: All_Spontaneous/2_con1.edf: the file is not EDF(+) or BDF(+) compliant (it contains format errors)


In [36]:
# getting trials for all mice
size = 36000
trials = np.zeros((len(dfs), int(dfs[0].shape[0]/size), size))

for i in range(trials.shape[0]):
    df = dfs[i]
    ts = df.values
    ts = np.reshape(ts, (ts.shape[1],ts.shape[0]))
    for j in range(int(df.shape[0]/size)):
        s = j*size
        trials[i,j,:] = ts[0,s:s+size]

In [37]:
# trials shorter than 24 hours
no24 = []
for i in range(len(names)):
    if dfs[i].shape[0]!= 51840000:
        no24.append(i)

1_con3_spont.edf: 12960000
1_KO2_spont.edf: 12960000
1_con2_spont.edf: 12960000
1_KO3_spont.edf: 12960000
2_KO1.edf: 12966000
2_KO3.edf: 12966000
2_con4.edf: 12966000
2_KO2.edf: 12966000
1_con1_spont.edf: 13020000
2_KO5.edf: 12960000
2_con2.edf: 12966000
2_con3.edf: 12966000
2_KO4.edf: 12966000
1_KO1_spont.edf: 13020000


In [38]:
# function for extracting frequency behavior from the data using a butterworth filter.
def bandpass_filter(data, lowcut, highcut, fs=600, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

In [41]:
# finding the mean frequency band power
def get_band_mean(data, low, high, fs = 600):
    # Band-pass filter
    filtered_data = bandpass_filter(data, low, high)
    
    # Compute the power (square of the amplitude)
    power = np.square(filtered_data)
    
    # Integrate the power over time using the trapezoidal rule
    integrated_power = np.trapz(power, dx=1/fs)
    
    # Compute the mean integrated power
    mean_integrated_power = integrated_power / len(data)
    
    return mean_integrated_power

# finding the peak frequency
def get_peak_frequency(data, low, high):
    # getting the frequncy band from data
    freqband = bandpass_filter(data, low, high)
    
    # finding the behavior of the frequnecy band
    frequencies, psd = welch(freqband, fs=600, nperseg=2048)
    
    # getting the peak
    peak = frequencies[np.argmax(psd)]
    return peak

In [53]:
# getting all the features for all the trials and mice
spont_trials = np.zeros((trials.shape[0],trials.shape[1],4)) # Theta mean - Gamma mean - Theta peak - Gamma peak

for i in tqdm(range(trials.shape[0])):
    if i in no24:
        for j in range(360):
            # Theta mean
            spont_trials[i,j,0] = get_band_mean(trials[i,j,:], 4, 10)
            # Gamma mean
            spont_trials[i,j,1] = get_band_mean(trials[i,j,:], 20, 80)
            # Theta peak
            spont_trials[i,j,2] = get_peak_frequency(trials[i,j,:], 4, 10)
            # Gamma peak
            spont_trials[i,j,3] = get_peak_frequency(trials[i,j,:], 20, 80)
        
    else:
        for j in range(trials.shape[1]):
            # Theta mean
            spont_trials[i,j,0] = get_band_mean(trials[i,j,:], 4, 10)
            # Gamma mean
            spont_trials[i,j,1] = get_band_mean(trials[i,j,:], 20, 80)
            # Theta peak
            spont_trials[i,j,2] = get_peak_frequency(trials[i,j,:], 4, 10)
            # Gamma peak
            spont_trials[i,j,3] = get_peak_frequency(trials[i,j,:], 20, 80)

100%|███████████████████████████████████████████| 31/31 [01:58<00:00,  3.82s/it]


In [79]:
# averaging across all trials for each mice
spont_results = np.zeros((trials.shape[0],8))

for i in range(trials.shape[0]):
    if i in no24:
        spont_results[i,:4] = np.mean(spont_trials[i,:360,:], axis=0)
        spont_results[i,4:] = np.std(spont_trials[i,:360,:], axis=0)
    else:
        spont_results[i,:4] = np.mean(spont_trials[i,:,:], axis=0)
        spont_results[i,4:] = np.std(spont_trials[i,:,:], axis=0)



In [80]:
# making a csv file
column_names = ['Theta Mean-mean', 'Gamma Mean-mean', 'Theta Peak-mean', 'Gamma Peak-mean', 'Theta Mean-std', 'Gamma Mean-std', 'Theta Peak-std', 'Gamma Peak-std']
spont_df = pd.DataFrame(spont_results, columns=column_names, index=names)

In [82]:
# seperating the Ko and control mice

spont_KO = spont_df[np.array(labels) == 1]
spont_con = spont_df[np.array(labels) == 0]

In [83]:
# saving to csv format

spont_KO.to_csv('spont_KO.csv')
spont_con.to_csv('spont_con.csv')